In [ ]:
import os

def sample_video(video_path, FRAME_RATE, path_output):
    # for filename in os.listdir(video_path):
    if video_path.endswith((".mp4", ".avi")):
        # filename = video_path + filename
        
        #os.system('ffmpeg -i \"{0}\" -vf fps={1} -q:v 2 \"{2}/frame_%05d.jpg\"'.format(video_path, FRAME_RATE,path_output))
        os.system('ffmpeg -i \"{0}\" -r {1} -q:v 2 \"{2}/frame_%05d.jpg\"'.format(video_path, FRAME_RATE,path_output))
    else:
        raise ValueError("Video path is not the name of video file (.mp4 or .avi)")

In [ ]:
import shutil

shutil.rmtree('/content/data')

In [ ]:
DATA_PATH = '/content/drive/MyDrive/종합설계/movies'
SAMPLE_DATA = '/content/data'
TRAIN_SAMPLE_DATA = '/content/data/train'
VALID_SAMPLE_DATA = '/content/data/valid'

if not os.path.exists(SAMPLE_DATA):
  os.mkdir(SAMPLE_DATA)
if not os.path.exists(TRAIN_SAMPLE_DATA):
  os.mkdir(TRAIN_SAMPLE_DATA)
if not os.path.exists(VALID_SAMPLE_DATA):
  os.mkdir(VALID_SAMPLE_DATA)

#train
for foldername in os.listdir(DATA_PATH):
  VIDEOS_PATH = os.path.join(TRAIN_SAMPLE_DATA, foldername)
  if not os.path.exists(VIDEOS_PATH):
    os.mkdir(VIDEOS_PATH)
  for videoname in os.listdir(os.path.join(DATA_PATH,foldername))[30:100]:
    video_path = DATA_PATH + '/' + foldername + '/' + videoname
    path_output = VIDEOS_PATH + '/' + videoname
    if not os.path.exists(path_output):
      os.mkdir(path_output)
    sample_video(video_path, 64, path_output)

In [ ]:
 
#valid
for foldername in os.listdir(DATA_PATH):
  VIDEOS_PATH = os.path.join(VALID_SAMPLE_DATA, foldername)
  if not os.path.exists(VIDEOS_PATH):
    os.mkdir(VIDEOS_PATH)
  for videoname in os.listdir(os.path.join(DATA_PATH,foldername))[:30]:
    video_path = DATA_PATH + '/' + foldername + '/' + videoname
    path_output = VIDEOS_PATH + '/' + videoname
    if not os.path.exists(path_output):
      os.mkdir(path_output)
    sample_video(video_path, 64, path_output)

model


In [ ]:
import os
os.chdir('/content/drive/MyDrive/종합설계/myslowfast')


In [ ]:
from dataset.dataset import DataGenerator
from model.model import SlowFast_body
from util import opts
from util.Callback import create_callbacks
from util.utils import get_optimizer
import tensorflow as tf
from model.SlowFast_pytorch import resnet50
import config

In [ ]:
opt = opts.opts()
#model = SlowFast_body(input_shape = (10,224,224,3),num_classes=2)
model = resnet50(class_num = 2)

optimizer = get_optimizer(opt)

train_data_generator = DataGenerator(opt.train_video_path, opt.file_path,opt.name_path, opt.data_name,
              'train', opt.batch_size, opt.num_classes, True,
              n_samples_for_each_video = opt.tau)

val_data_generator = DataGenerator(opt.valid_video_path, opt.file_path,opt.name_path, opt.data_name,
              'val', opt.batch_size, opt.num_classes, False,
              n_samples_for_each_video = opt.tau)

callback = create_callbacks(opt,max(1, train_data_generator.__len__()))

In [ ]:
#model.load_weights('/content/drive/MyDrive/종합설계/myslowfast/result/trained_weights_final.h5')
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
### keras version
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Train model on datase·t
history=None

with tf.device('/device:GPU:0'):
  history=model.fit_generator(generator=train_data_generator,
                      steps_per_epoch=max(1, train_data_generator.__len__()),
                      validation_data=val_data_generator, 
                      validation_steps=max(1, val_data_generator.__len__()),
                      epochs=30,
                      callbacks = callback)

#history.save_weights('/content/drive/MyDrive/종합설계/myslowfast/result/trained_weights_final.h5')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Plotting 
acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']
plt.figure(figsize=(8, 8)) 
plt.subplot(2, 1, 1) 
plt.plot(acc, label='Training Accuracy') 
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right') 
plt.ylabel('Accuracy') 
plt.ylim([min(plt.ylim()),1]) 
plt.title('Training and Validation Accuracy') 
plt.subplot(2, 1, 2) 
plt.plot(loss, label='Training Loss') 
plt.plot(val_loss, label='Validation Loss') 
plt.legend(loc='upper right') 
plt.ylabel('Cross Entropy') 
plt.ylim([0,1.0]) 
plt.title('Training and Validation Loss') 
plt.xlabel('epoch') 
plt.show()



실험

In [ ]:
#test_video = '/content/data/Violence/crowd_violence__Roma_V_Man_Utd_Police_beating_crowd__Beck84Utd__jwtg8AkcAfc.avi'
from PIL import Image
import numpy as np
for folder in os.listdir('/content/data'):
  path = '/content/data' + '/' + folder
  for test_video in os.listdir(path)[::-1]:
    test = []
    for image in os.listdir(path + '/' + test_video)[:64]:
      img = Image.open(path + '/' + test_video + '/' + image)
      img = img.resize((224,224))
      _np = np.array(img)
      test.append(_np)
    test = np.array(test)
    test = np.expand_dims(test,0)
    print(model.predict(test))

In [ ]:
import tensorflow_datasets as tfds

tfds.load()

In [ ]:
from keras.models import load_model
model1 = load_model('/content/drive/MyDrive/종합설계/myslowfast/resnet50_imagenet.h5')

In [ ]:
model2 = load_model('/content/drive/MyDrive/종합설계/myslowfast/resnet50_places365.h5')

In [ ]:
model1.summary()

In [ ]:
model2.summary()

In [ ]:
print(len(model1.layers), len(model2.layers))

In [ ]:
tt = ResNet50(include_top=True)

In [ ]:
len(tt.layers)

In [ ]:
!wget --no-check-certificate https://www.crcv.ucf.edu/datasets/human-actions/ucf101/UCF101.rar

In [ ]:
!unrar x UCF101.rar

In [ ]:
from __future__ import print_function, division
import os
import sys
import subprocess

def class_process(dir_path, dst_dir_path, class_name):
  class_path = os.path.join(dir_path, class_name)
  if not os.path.isdir(class_path):
    return

  dst_class_path = os.path.join(dst_dir_path, class_name)
  if not os.path.exists(dst_class_path):
    os.mkdir(dst_class_path)

  for idx,file_name in enumerate(os.listdir(class_path)):
    if '.avi' not in file_name and '.mp4' not in file_name:
      continue
    name, ext = os.path.splitext(file_name)
    dst_directory_path = os.path.join(dst_class_path, name)

    video_file_path = os.path.join(class_path, file_name)
    try:
      if os.path.exists(dst_directory_path):
        if not os.path.exists(os.path.join(dst_directory_path, 'image_00001.jpg')):
          subprocess.call('rm -r \"{}\"'.format(dst_directory_path), shell=True)
          print('remove {}'.format(dst_directory_path))
          os.mkdir(dst_directory_path)
        else:
          continue
      else:
        os.mkdir(dst_directory_path)
    except:
      print(dst_directory_path)
      continue
    cmd = 'ffmpeg -i \"{}\" -vf scale=-1:240 \"{}/image_%05d.jpg\"'.format(video_file_path, dst_directory_path)
    if idx%10==0:
      print(str(idx) + '\n')
    subprocess.call(cmd, shell=True)
    

if __name__=="__main__":
  dir_path = 'UCF-101'
  dst_dir_path = 'UCF101_jpg'

  if not os.path.exists(dst_dir_path):
    os.mkdir(dst_dir_path)

  for class_name in os.listdir(dir_path):
    class_process(dir_path, dst_dir_path, class_name)

In [ ]:
from __future__ import print_function, division
import os
import sys
import subprocess

def class_process(dir_path, class_name):
  class_path = os.path.join(dir_path, class_name)
  if not os.path.isdir(class_path):
    return

  for file_name in os.listdir(class_path):
    video_dir_path = os.path.join(class_path, file_name)
    image_indices = []
    for image_file_name in os.listdir(video_dir_path):
      if 'image' not in image_file_name:
        continue
      image_indices.append(int(image_file_name[6:11]))

    if len(image_indices) == 0:
      print('no image files', video_dir_path)
      n_frames = 0
    else:
      image_indices.sort(reverse=True)
      n_frames = image_indices[0]
      print(video_dir_path, n_frames)
    with open(os.path.join(video_dir_path, 'n_frames'), 'w') as dst_file:
      dst_file.write(str(n_frames))


if __name__=="__main__":
  dir_path = '/content/UCF101_jpg'
  for class_name in os.listdir(dir_path):
    class_process(dir_path, class_name)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/종합설계/myslowfast')
from dataset.dataset import DataGenerator
from model.model import SlowFast_body
from util import opts
from util.Callback import create_callbacks
from util.utils import get_optimizer
import tensorflow as tf

opt = opts.opts()

optimizer = get_optimizer(opt)


train_data_generator = DataGenerator('/content/UCF101_jpg', 'dataset/ucfTrainTestlist/trainlist01.txt', 'dataset/ucfTrainTestlist/classInd.txt', 'ucf101',
              'train', opt.batch_size, 101, True,
              n_samples_for_each_video = opt.tau)

val_data_generator = DataGenerator('/content/UCF101_jpg', 'dataset/ucfTrainTestlist/testlist01.txt', 'dataset/ucfTrainTestlist/classInd.txt', 'ucf101',
              'val', opt.batch_size, 101, False,
              n_samples_for_each_video = opt.tau)

callback = create_callbacks(opt,max(1, train_data_generator.__len__()))

In [ ]:
#model.load_weights('/content/drive/MyDrive/종합설계/myslowfast/result/trained_weights_final.h5')

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
model = SlowFast_body(input_shape = (32,224,224,3),num_classes=101)

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Train model on datase·t
history=None

with tf.device('/device:GPU:0'):
  history=model.fit_generator(train_data_generator,
                      steps_per_epoch=max(1, train_data_generator.__len__()),
                      validation_data=val_data_generator, 
                      validation_steps=max(1, val_data_generator.__len__()),
                      epochs=30,
                      callbacks = callback)

#history.save_weights('/content/drive/MyDrive/종합설계/myslowfast/result/trained_weights_final.h5')
